# MLP Architecture Comparison UI

Use the controls below to set the sample size and MLP architectures. The baseline `[]` is always included. Click **Run** to train, compare, and visualize FP/FN overlays.

In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt

from data import generate_intertwined_spirals, plot_three_datasets_with_fp_fn
from MLPx6 import (
    MLP,
    MLP_ARCHITECTURES,
    prepare_data,
    train_model,
    evaluate_model,
    confusion_counts,
    plot_learning_curves,
 )

try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
except ImportError:
    widgets = None
    display = None
    clear_output = None


def parse_custom_arch(text):
    if not text:
        return None
    parts = [p.strip() for p in text.split(",") if p.strip()]
    try:
        values = [int(p) for p in parts]
    except ValueError:
        return None
    return values if values else None


def collect_fp_fn(model, test_loader):
    model.eval()
    fp_points = []
    fn_points = []
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            outputs = model(x_batch)
            predicted = (outputs > 0.5).float()
            fp_mask = (predicted == 1) & (y_batch == 0)
            fn_mask = (predicted == 0) & (y_batch == 1)
            if fp_mask.any():
                fp_xy = x_batch[fp_mask.squeeze()]
                fp_points.extend([(float(x), float(y)) for x, y in fp_xy.tolist()])
            if fn_mask.any():
                fn_xy = x_batch[fn_mask.squeeze()]
                fn_points.extend([(float(x), float(y)) for x, y in fn_xy.tolist()])
    return fp_points, fn_points


def compute_metrics(tn, fp, fn, tp):
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0
    f1 = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0.0
    return precision, recall, specificity, f1

ModuleNotFoundError: No module named 'torch'

In [2]:
import sys, subprocess, importlib.util, json, os

# 1️⃣ Show which interpreter we are using
print("Using:", sys.executable)

# 2️⃣ Install ipywidgets if missing
if importlib.util.find_spec("ipywidgets") is None:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ipywidgets"])

# 3️⃣ Verify import works
import ipywidgets as widgets
print("ipywidgets version:", widgets.__version__)

# 4️⃣ Quick sanity‑check widget
from IPython.display import display
display(widgets.IntSlider(value=3, min=0, max=10, description="Demo:"))if "widgets" not in globals() or widgets is None:
    try:
        import ipywidgets as widgets
        from IPython.display import display, clear_output
    except ImportError:
        widgets = None
        display = None
        clear_output = None

if widgets is None:
    print("ipywidgets is not installed. Install it to use the UI.")
else:
    arch_options = [
        (f"{name}: {arch}", name)
        for name, arch in MLP_ARCHITECTURES.items()
        if name != "MLP_0"
    ]

    n_input = widgets.IntText(value=1000, description="Samples", layout=widgets.Layout(width="180px"))
    dataset_input = widgets.Dropdown(
        options=["RND", "CTR", "EDGE"], value="RND", description="Dataset"
    )
    epochs_input = widgets.IntText(value=100, description="Epochs", layout=widgets.Layout(width="180px"))
    lr_input = widgets.FloatText(value=0.001, description="LR", layout=widgets.Layout(width="180px"))
    batch_input = widgets.IntText(value=32, description="Batch", layout=widgets.Layout(width="180px"))
    arch_select = widgets.SelectMultiple(
        options=arch_options,
        value=(arch_options[0][1],),
        description="Architectures",
        layout=widgets.Layout(width="380px", height="140px"),
    )
    custom_arch_input = widgets.Text(
        value="", description="Custom", placeholder="e.g., 32,32,16"
    )
    run_button = widgets.Button(description="Run", button_style="primary")
    output = widgets.Output()

    controls_row1 = widgets.HBox([n_input, dataset_input, epochs_input, lr_input, batch_input])
    controls_row2 = widgets.HBox([arch_select, custom_arch_input])
    display(controls_row1, controls_row2, run_button, output)

    def on_run(_):
        with output:
            clear_output(wait=True)
            n = int(n_input.value)
            dataset_choice = dataset_input.value
            epochs = int(epochs_input.value)
            lr = float(lr_input.value)
            batch_size = int(batch_input.value)
            selected_arch_names = list(arch_select.value)
            custom_arch = parse_custom_arch(custom_arch_input.value.strip())
            if custom_arch_input.value.strip() and custom_arch is None:
                print("Invalid custom architecture; ignoring.")
            baseline_arch = ("BASELINE", [])
            selected_arch = [
                (name, MLP_ARCHITECTURES[name])
                for name in selected_arch_names
                if name in MLP_ARCHITECTURES
            ]
            if custom_arch is not None:
                selected_arch.append(("CUSTOM", custom_arch))
            if not selected_arch:
                print("No architectures selected; using baseline only.")
            torch.manual_seed(7)
            np.random.seed(7)

            rnd_data, ctr_data, edge_data = generate_intertwined_spirals(
                n=n, noise_std=0.0, seed=7, sampling_method="ALL", plot=False
            )
            dataset_storage = {"RND": rnd_data, "CTR": ctr_data, "EDGE": edge_data}
            train_loader, val_loader, test_loader = prepare_data(
                dataset_storage[dataset_choice],
                test_split=0.2,
                val_split=0.2,
                batch_size=batch_size,
            )

            architectures_to_train = [baseline_arch] + selected_arch
            train_histories = {}
            results = []
            for name, hidden_layers in architectures_to_train:
                model = MLP(hidden_layers)
                trained_model, train_losses, val_losses, test_losses = train_model(
                    model, train_loader, val_loader, test_loader, epochs=epochs, lr=lr, verbose=False
                )
                acc = evaluate_model(trained_model, test_loader)
                tn, fp, fn, tp = confusion_counts(trained_model, test_loader)
                precision, recall, specificity, f1 = compute_metrics(tn, fp, fn, tp)
                total_params = sum(param.numel() for param in trained_model.parameters())
                train_histories[name] = {"train_losses": train_losses, "test_losses": test_losses}
                results.append({
                    "name": name,
                    "arch": hidden_layers,
                    "acc": acc,
                    "tn": tn,
                    "fp": fp,
                    "fn": fn,
                    "tp": tp,
                    "precision": precision,
                    "recall": recall,
                    "specificity": specificity,
                    "f1": f1,
                    "total_params": total_params,
                    "model": trained_model,
                })

            print("=" * 60)
            print("RUN PARAMETERS")
            print("=" * 60)
            print(f"Dataset: {dataset_choice}")
            print(f"n per region: {n} | batch: {batch_size} | epochs: {epochs} | lr: {lr}")
            print("Architectures:")
            for item in results:
                print(f"  {item['name']}: {item['arch']}")

            print("\n" + "=" * 60)
            print("MODEL RESULTS")
            print("=" * 60)
            for item in results:
                print(f"{item['name']}: arch={item['arch']} | params={item['total_params']}")
                print(
                    f"  Acc={item['acc']*100:.2f}% | "
                    f"TN={item['tn']} FP={item['fp']} FN={item['fn']} TP={item['tp']}"
                )
                print(
                    f"  Precision={item['precision']:.3f} Recall={item['recall']:.3f} "
                    f"Specificity={item['specificity']:.3f} F1={item['f1']:.3f}"
                )

            plot_learning_curves(train_histories, title=f"Training Loss - {dataset_choice}")
            best_result = max(results, key=lambda r: r["acc"])
            fp_points, fn_points = collect_fp_fn(best_result["model"], test_loader)
            fp_fn_by_dataset = {
                dataset_choice: {
                    "fp": fp_points,
                    "fn": fn_points,
                    "model_name": best_result["name"],
                    "acc": best_result["acc"],
                }
            }
            plot_three_datasets_with_fp_fn(dataset_storage, fp_fn_by_dataset)

    run_button.on_click(on_run)

SyntaxError: invalid syntax (845995401.py, line 17)

In [5]:
# UI-driven run happens in the cell above.